#### **Deep & Reinforcement Learning**

#### **Observações**

##### Classificando diferentes tipos de semente de abóbora 🎃🌱

As sementes de abóbora são frequentemente consumidas como confeitos em todo o mundo devido à sua quantidade adequada de proteínas, gorduras, carboidratos e teores minerais. A base de dados **"SementesAbobora.xlsx"** possui um estudo foi realizado nos dois tipos de sementes de abóbora mais importantes e de qualidade, **“Ürgüp Sivrisi”** e **“Çerçevelik”**, geralmente cultivadas nas regiões de Ürgüp e Karacaören na Turquia.

Muitas espécies de sementes têm semelhanças visuais, o que torna a classificação manual difícil e sujeita a erros. Redes neurais podem ser treinadas para identificar padrões que não são facilmente perceptíveis pelo olho humano, aumentando a precisão da classificação.

Imagine que foi proposto para você o desafio de criar uma **inteligência para identificar os tipos de sementes para ajudar a equipe de engenheiros e engenheiras Agrícolas**. Para trabalhar com a precisão dos resultados x complexidade das características de sementes, você optou em utilizar as **redes neurais multilayer perceptron**. Vamos para a aplicação?

**Features**

- Perímetro
- Maior_Eixo_Comprimento
- Comprimento_Eixo_Menor
- Área_Convexa
- Equiv_Diâmetro
- Excentricidade
- Solidez
- Extensão
- Redondeza
- Proporcao
- Compacidade

**Target**
Classes:
 ((A)Çerçevelik, (B)Ürgüp Sivrisi)

#### **Conteúdo - Bases e Notebook da aula**

Github:  
https://github.com/FIAP/Pos_Tech_DTAT/tree/DeepLearning


#### **Importação de bibliotecas**

In [107]:
#Importar biblioteca completa
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

#Importar algo especifico de uma biblioteca
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# keras libraries
from tensorflow import keras
from keras import models
from keras import layers
from keras import metrics
from keras.optimizers import Adam
from keras.metrics import Precision
from tqdm.keras import TqdmCallback

ModuleNotFoundError: No module named 'tensorflow'

#### **Funções (def)**

#### **Aula 1 - Perceptron de múltiplas camadas**

In [ ]:
#Importar base de dados

link = 'https://github.com/RicardViana/fiap-Deep-Learning-and-unstructured-data/raw/refs/heads/main/SementesAbobora.xlsx'
df = pd.read_excel(link)

In [ ]:
#Ver os primeiros dados

df.head()

In [ ]:
#Ver os ultimos dados

df.tail()

In [ ]:
#Ver a qtd de linhas e colunas da data frame
df.shape

In [ ]:
#Como a base de dados possui os dados de classes de forma ordenada, isso pode ser um problema para o aprendizado de máquina e dessa forma é preciso embaralhar os dados com o comando shuffle.

'''

Se o que você quer prever é uma categoria (nome, grupo, "sim/não"), você está lidando com Classes.

Se o que você quer prever é um número contínuo (preço de uma casa, temperatura, valor do dólar), você está lidando com Regressão.

'''

df = shuffle(df)

In [ ]:
#Verificar os dados embaralhados
df.head()

In [ ]:
#Ver o equilibrio da base de dados
df['Class'].value_counts()

In [ ]:
#Potando histogramas para analisar a simetria dos dados
df.hist(bins=100, figsize=(12, 12))
plt.show()

In [ ]:
#Verificar a correlação dos dados
correlation_matrix = df.select_dtypes(include='number').corr().round(2)

fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data=correlation_matrix, annot=True, linewidths=.5, ax=ax)
plt.show()

In [ ]:
#Ver a info dos dados
df.info()

In [ ]:
# Utilizando LabelEncoder para transformar/converter rótulos de texto em números referente a variavel target
le = LabelEncoder()

# Sobrescrever a coluna Class
df.Class = le.fit_transform(df['Class'])

In [ ]:
# Verificar apenas os dados distitos
set(df['Class'])

In [ ]:
# Separar os dados 
X = df[['Area','Perímetro', 'Comprimento_Eixo_Menor','Excentricidade','Solidez','Extensão','Redondeza', 'Proporcao', 'Compacidade']]
y = df['Class']

In [ ]:
# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# Pré-processamento dos dados

'''

Apesar de deep learning não requerer distribuições normais, é comum aplicar técnicas de pré-processamento de dados para normalizar ou padronizar as características. 
Isso pode ajudar a acelerar a convergência do treinamento da rede neural.
A convergência está relacionada com o erro, ou seja, o quanto a sua rede aprende a corrigir os erros durante o processamento.

'''

# Ajustar os dados (Fit)
scaler = StandardScaler()
scaler.fit(X_train)
scaler.fit(X_test)

In [ ]:
# Transformar os dados
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Ver os dados de treino
X_train

In [ ]:
# Ver a quantidade de linhas e colunas da base de treino
X_train.shape

In [ ]:
# Ver o tipo do dado
type(X_train)

In [ ]:
# Ver a quantidade de linhas e colunas da base de teste
X_test.shape

In [ ]:
# Ver o tipo do dado
type(X_test)

In [ ]:
#Moldando nossas variáveis resposta

'''

A nossas classes estão em um formato de estrutura de dados unidimensional (assim como uma coluna) e precisamos moldar para o formato de array -n dimensional(matriz).
O comando reshape é utilizado para reformatar a estrutura de um array multidimensional, como um tensor, que é a estrutura de dados fundamental usada em deep learning e processamento de dados em redes neurais.

'''

# Ver o tipo do dado
type(y_train)

In [ ]:
# reshape() molda uma matriz sem alterar os dados da matriz

'''

O que o .reshape((-1, 1)) faz?

Imagine que os dados originais sejam uma lista simples: [0, 1, 0, 1] e para o Python, isso é um vetor de uma única dimensão (1D). 
No entanto, bibliotecas como o TensorFlow ou Scikit-Learn geralmente exigem que os dados de entrada tenham duas dimensões: uma para as linhas (amostras) e outra para as colunas (características).

* O número 1: Diz ao NumPy que queremos que o resultado final tenha exatamente 1 coluna.
* O número -1: Esse é um "coringa". Ele diz ao NumPy: "Calcule automaticamente quantas linhas são necessárias para que todos os dados caibam nessa 1 coluna".

O passo a passo da linha inteira

O comando abaixoé uma "super instrução" que faz três coisas:

1. np.asarray(y_train): Garante que o dado seja um array do NumPy (caso ele venha de uma lista ou de uma coluna do Pandas).
2. .astype('float32'): Converte os números para o formato de ponto flutuante de 32 bits. Computacionalmente, as GPUs e CPUs processam cálculos de Redes Neurais muito mais rápido em float32 do que em números inteiros ou float64.
3. .reshape((-1, 1)): Transforma o vetor de "deitado" (1D) para "em pé" (2D), como uma coluna de uma tabela.

Por que isso é necessário?

A maioria dos modelos de Deep Learning (como os do Keras/TensorFlow) espera que o seu `y` (o que você quer prever) tenha o formato (número_de_amostras, número_de_saídas).

Mesmo que você só tenha uma saída (0 ou 1), o modelo espera que essa saída esteja dentro de uma estrutura de "coluna", e não apenas uma lista solta.

'''

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
# Ver o tipo do dado após o reshape
type(y_train)

In [ ]:
# Ver a qtd de linhas e colunas
y_train.shape

In [ ]:
# Construindo a arquitetura da rede neural multicamadas 

'''

Agora utilizando as bibliotecas tensorflow e keras, vamos construir a arquitetura da nossa primeira rede neural

'''

#### **Aula 2 - Redes convolucionais**

#### **Aula 3 - Transferência de aprendizagem**

#### **Aula 4 - Redes recorrentes**

#### **Aula 5 - Redes não supervisionadas**

#### **Aula 6 - Gans**